# Related Artist Tracks
Now that I can pull all tracks for an artist and I have a simple recommender system that can find similar tracks to the ones I've already liked, I want to crawl through the similar artists to mine and compare their tracks to the ones I've liked to find new, similar tracks to recommend.

Starting with my "Tom's Funky Playlist" I'll get a unique list of artists.

In [1]:
from spoti_pandas import sp_functions as sppd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

# # Spotify Credentials
# import spot_creds

# clid = spot_creds.client_id
# secret = spot_creds.secret

# #Authentication - without user
# client_credentials_manager = SpotifyClientCredentials(client_id=clid, client_secret=secret)
# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [2]:
pl_link = '"https://open.spotify.com/playlist/7eWWLoTfmLUcD0viBP6Hr0?si=e8b0760749404749"' # Tom's Funky Playlist
pl_tracks = sppd.pl_track_features(pl_link)
pl_tracks.head()

,track_uri,track_name,artist_uri,artist_name,artist_pop,artist_genres,album,track_pop,explicit,acousticness,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,spotify:track:5ZDLCdSHmpt2JKwWmotjzG,Pants,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,34,[funk rock],Single Entendre,0,False,0.02650,...,0.851,0.000000,1,0.3580,-4.067,0,0.0351,123.910,4,0.809
1,spotify:track:6NBqMK4DfBYGIzCnmSgliG,Dirty Minds,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,34,[funk rock],Everlasting Party,0,False,0.00999,...,0.832,0.000000,9,0.0902,-5.705,1,0.0605,132.901,4,0.878
2,spotify:track:4zxXklp6D4vClyC08SUeo9,Carnal Carnival,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,34,[funk rock],Carnal Carnival,0,False,0.44300,...,0.934,0.000000,11,0.6820,-3.049,0,0.0708,160.061,4,0.753
3,spotify:track:3qaO2Ko1b0bBisqNLU1e1s,Freak Flag,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,34,[funk rock],Carnal Carnival,0,False,0.03550,...,0.901,0.000000,1,0.3740,-3.393,1,0.0452,109.968,4,0.830
4,spotify:track:6zADCSrbXuHSTnZbznyyY9,Live in the Moment,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,34,[funk rock],Live in the Moment,0,False,0.00324,...,0.753,0.000215,1,0.1740,-6.065,0,0.0329,105.947,4,0.502


In [3]:
pl_tracks.columns

Index(['track_uri', 'track_name', 'artist_uri', 'artist_name', 'artist_pop',
       'artist_genres', 'album', 'track_pop', 'explicit', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature',
       'valence'],
      dtype='object')

In [4]:
pl_tracks.artist_name.unique()

array(['Here Come The Mummies', 'Mingo Fishtrap', 'Phat Phunktion',
       'Prince', 'Sister Sparrow', 'Santa Fe & The Fat City Horns',
       'Jaco Pastorius', 'MF Robots', 'Philip Lassiter', 'Thomas Atlas',
       'Orgone', 'Tom McGuire & the Brassholes', 'The Fearless Flyers',
       'Lawrence', 'Tower Of Power', "Home Cookin'", 'Cory Wong',
       'David Sanborn', 'Candy Dulfer', 'The Jazz Crusaders', 'Incognito',
       'Maceo Parker'], dtype=object)

In [5]:
pl_artists = pl_tracks.artist_uri.unique()
pl_artists

array(['spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L',
       'spotify:artist:7mYdzBqQUa1hb8jdRmMpfE',
       'spotify:artist:3afwdwY97SMtIdnUsPTpQT',
       'spotify:artist:5a2EaR3hamoenG9rDuVn8j',
       'spotify:artist:5kah03Pzp6s8eD8vRIhdfL',
       'spotify:artist:0DJRN6LYpYlKu30dsXefJv',
       'spotify:artist:3U3C9o6UTYNdEsDckpRyvX',
       'spotify:artist:3gfBx0SvMGdMQ2ZsjPvIV4',
       'spotify:artist:1wnaeDbP5Yl9MNV9qC008L',
       'spotify:artist:6JmjdmUmKCrzq0CqDt0vB6',
       'spotify:artist:4WmMnGO1nLIsE85XwcBAZE',
       'spotify:artist:7JnJgTo8cCtAQmtC0cJyjp',
       'spotify:artist:1JyLSGXC3aWzjY6ZdxvIXh',
       'spotify:artist:5rwUYLyUq8gBsVaOUcUxpE',
       'spotify:artist:0JCxGVxsISZzJHJPUOtceB',
       'spotify:artist:4GAyr0SBHrn2nL4tvwASuP',
       'spotify:artist:6xt9sJmmyYwWkJv8A6ssiU',
       'spotify:artist:7vNcTk9TgKF0qDsS87nWGE',
       'spotify:artist:287jMoxHzjERgHI6ja8TKa',
       'spotify:artist:6BufIg68WvvzrEqoDI19Kl',
       'spotify:artist:5moJNCJeiNwuQAhDL

## Find Related Artists
For all of the playlist artists, find the related artists and create a unique list.

In [6]:
related_artists = pd.DataFrame()
for artist in pl_artists:
    these_related = sppd.related_artists(artist)
    related_artists = pd.concat([related_artists, these_related])
related_artists.drop_duplicates('name',  inplace=True)

In [7]:
len(related_artists)

329

In [8]:
related_artists.sort_values('popularity', ascending=False, inplace=True)

## Get all Tracks for Related Artists

In [ ]:
related_tracks = pd.DataFrame()

for i in range(len(related_artists)):
    artist_name = related_artists.iloc[i]['name']
    artist_uri = related_artists.iloc[i]['uri']
    print(f'{i}/{len(related_artists)} {artist_name}: {artist_uri}')
    
    this_artist_tracks = sppd.get_artist_track_features(artist_uri)
    print(f'{len(this_artist_tracks)} tracks found')
    related_tracks = pd.concat([related_tracks, this_artist_tracks])
    
    #write to excel in case of crash or rate-limit
    related_tracks.to_excel("funky_related_tracks.xlsx", index=False)
    #sleep to avoid rate limit
    time.sleep(60)

0/329 Bootsy Collins: spotify:artist:5K0rbdBrs2tNXe5LeWMATT
186 tracks found
1/329 Janet Jackson: spotify:artist:4qwGe91Bz9K2T8jXTZ815W
280 tracks found
2/329 Commodores: spotify:artist:6twIAGnYuIT1pncMAsXnEm
230 tracks found


In [ ]:
len(related_tracks)

In [ ]:
50 % 50